## WITS TradeStats (World Bank)
*Client* mínimo para puxar trade flows por parceiro.

Foco do exemplo:
- Importação e exportação de Ureia (HS6=310210) da Índia (reporter=ind) com todos os parceiros (partner=all).
- Indicadores: MPRT-TRD-VL (import value) e XPRT-TRD-VL (export value).

Referências (API oficial):
- Base URL: `https://wits.worldbank.org/API/V1`
- TradeStats data request (SDMX/V21/datasource/...): ver [WITS API User Guide](https://wits.worldbank.org/data/public/WITSAPI_UserGuide.pdf).

In [ ]:
import time
from dataclasses import dataclass
from typing import Iterable, Literal, Optional
import io
import pandas as pd
import requests
import numpy as np
import os

TradeFlow = Literal["Imports", "Exports"]